### 1. Database schema tables (Using 3NF):
- Table 1: ```Symbols```
    - Symbol (Type: ```Text```) **(Primary Key)**
    - Name (Type: ```Text```)
    - Industry (Type: ```Text```)
    
- Table 2: ```Membership```
    - (Symbol, ReportDate) **(Composite Primary Key)**
        - Symbol (Type: ```Text```)
        - ReportDate (Type: ```DateTime```)
    - Weightage (Type: ```Double```)

- Table 3: ```Attributes```
    - (Symbol, ReportDate) **(Composite Primary Key)**
        - Symbol (Type: ```Text```)
        - ReportDate (Type: ```DateTime```)
    - Equity Capital (In Rs.) (Type: ```Double```)	
    - Free Float Market Capitalisation (Rs. Crores) (Type: ```Double```)	
    - Beta (Type: ```Double```)	
    - R2 (Type: ```Double```)	
    - Volatility_Per (Type: ```Double```)	
    - Monthly Return (Type: ```Double```)	
    - Avg_Impact_Cost_percent (Type: ```Double```)
    
<br />

The design approach here is to use normalisation approach. In any normalisation approach, we typically look at 1NF, 2NF, 3NF
- 1NF: Information in each row is atomic
- 2NF: One column is dependent on combination of columns 
- 3NF: Each column is dependent on only the primary key

As for the constraints of the database schema, please see the above for Type of each columns and the primary keys. In addition, the foreign key of each tables are as follows:
- ```Membership```
    - Symbol as a Foreign Key to ```Symbols```
- ```Attributes```:
    - Symbol as a Foreign Key to ```Symbols```
    - ReportDate as a Foreign key to ```Membership```

However, I choose 3NF over 1NF and 2NF as it provides the following benefits:
- Reducing data redundancy by allowing each column to only depend on primary key (3NF)
- More modularity and organised
- Better query performance







### 2. QC checks before populating into DB schema

In [2]:
# Import libraries
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
pd.set_option('display.max_rows', 500)
filepath = "nifty_data.xlsx"  # Change this accordingly


# Handle duplicated records since they are "double counted" in Nifty50 per each membership date
raw = pd.read_excel(filepath)
raw = raw.drop_duplicates(keep='first')  


# Clean the symbols (ID_EXCH_SYMBOL EX005) 
print("Checking for any wrong symbols in raw dataset", sorted(raw['Symbol'].unique())) # 'BAJAJ-AUTO', 'BAJAJAUTO' is wrong from the list
raw = raw.replace({'Symbol': {'BAJAJAUTO': 'BAJAJ-AUTO'}})


# Keep golden copy of security master reference dataset based on unique Symbols to clean our dataset later
# Replace missing records of name and industry based on the symbols
print("Checking for columns with missing records in raw dataset", raw.columns[raw.isnull().any()].tolist())  # Name and Industry columns are missing records
Symbols = raw[['Symbol', 'Name', 'Industry']].drop_duplicates()  
Symbols = Symbols.dropna(how = "all")  # Remove rows with all NAs in them
Symbols['Name'] = Symbols.groupby('Symbol')['Name'].transform(lambda x: x.bfill().ffill())  # Populate missing Name based on Symbol
Symbols['Industry'] = Symbols.groupby('Symbol')['Industry'].transform(lambda x: x.bfill().ffill()) # Populate missing Industry based on Symbol
Symbols = Symbols.drop_duplicates(subset="Symbol")


# Clean up GLAXO, REL and VSNL whose Industry is in Name col and are missing Industry -> swap values in industry and name around
# Check per NSEIndia website the proper names of those symbols: https://www.nseindia.com/get-quotes
symbols_to_swap = ['GLAXO', 'REL', 'VSNL']
mask = Symbols['Symbol'].isin(symbols_to_swap)
Symbols.loc[mask, ['Name', 'Industry']] = Symbols.loc[mask, ['Industry', 'Name']].values
correct_names = {'GLAXO': 'GlaxoSmithKline', 'REL': 'Reliance', 'VSNL': 'VSNL Broadband'}
mask = Symbols['Symbol'].isin(correct_names.keys())
Symbols.loc[mask, 'Name'] = Symbols.loc[mask, 'Symbol'].map(correct_names)


# Clean up our original dataframe with the correct Name and Industry
cleaned = pd.merge(raw, Symbols, on='Symbol', how='left')
cleaned.drop(['Name_x', 'Industry_x'], axis=1, inplace=True)
cleaned.rename(columns={'Name_y': 'Name', 'Industry_y': 'Industry'}, inplace=True)


# Convert to correct data types for those who are not the same as our DB schema
cleaned['Monthly Return'] = pd.to_numeric(cleaned['Monthly Return'], errors='coerce')  # Monthly Returns is in string format
correct_datatypes = {
      'Symbol': "object",
   'Equity Capital (In Rs.)': "float64",
   'Free Float Market Capitalisation (Rs. Crores)': "float64",
   'Weightage ()': "float64",
   'Beta': "float64",
   'R2': "float64",
   'Volatility_Per': "float64",
   'Monthly Return': "float64",
   'Avg_Impact_Cost_percent': "float64",
   'report_date': "datetime64[ns]",
   'Name': "object",
   'Industry': "object"
   }
cleaned = cleaned.astype(correct_datatypes)
cleaned.head()


Checking for any wrong symbols in raw dataset ['ABB', 'ACC', 'ADANIPORTS', 'AMBUJACEM', 'ASIANPAINT', 'AUROPHARMA', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJAUTO', 'BAJAJFINSV', 'BAJFINANCE', 'BANKBARODA', 'BHARTIARTL', 'BHEL', 'BOSCHLTD', 'BPCL', 'BRITANNIA', 'CAIRN', 'CIPLA', 'COALINDIA', 'DLF', 'DRREDDY', 'EICHERMOT', 'GAIL', 'GLAXO', 'GRASIM', 'HCLTECH', 'HDFC', 'HDFCBANK', 'HEROHONDA', 'HEROMOTOCO', 'HINDALCO', 'HINDPETRO', 'HINDUNILVR', 'IBULHSGFIN', 'ICICIBANK', 'IDEA', 'IDFC', 'INDUSINDBK', 'INFOSYSTCH', 'INFRATEL', 'INFY', 'IOC', 'ITC', 'JINDALSTEL', 'JPASSOCIAT', 'JSWSTEEL', 'KOTAKBANK', 'LT', 'LUPIN', 'M&M', 'MARUTI', 'MCDOWELL-N', 'NATIONALUM', 'NESTLEIND', 'NMDC', 'NTPC', 'ONGC', 'PNB', 'POWERGRID', 'RANBAXY', 'RCOM', 'REL', 'RELCAPITAL', 'RELIANCE', 'RELINFRA', 'RPL', 'RPOWER', 'SAIL', 'SATYAMCOMP', 'SBIN', 'SESAGOA', 'SIEMENS', 'SSLT', 'STER', 'SUNPHARMA', 'SUZLON', 'TATACOMM', 'TATAMOTORS', 'TATAMTRDVR', 'TATAPOWER', 'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'UNITECH',

,Symbol,Equity Capital (In Rs.),Free Float Market Capitalisation (Rs. Crores),Weightage (),Beta,R2,Volatility_Per,Monthly Return,Avg_Impact_Cost_percent,report_date,Name,Industry
0,ABB,4.238168e+08,7700.0,0.60,0.88,0.52,2.09,8.29,0.09,2009-08-01,ABB Ltd.,ELECTRICAL EQUIPMENT
1,ACC,1.876949e+09,8148.0,0.64,0.73,0.42,2.77,-8.38,0.08,2009-08-01,ACC Ltd.,CEMENT AND CEMENT PRODUCTS
2,AMBUJACEM,3.045618e+09,8084.0,0.63,0.84,0.41,1.94,-8.62,0.12,2009-08-01,Ambuja Cements Ltd.,CEMENT AND CEMENT PRODUCTS
3,AXISBANK,3.597637e+09,18826.0,1.47,1.24,0.61,2.64,-1.29,0.07,2009-08-01,Axis Bank Ltd.,BANKS
4,BHARTIARTL,3.796650e+10,52544.0,4.11,0.99,0.62,2.64,3.54,0.09,2009-08-01,Bharti Airtel Ltd.,TELECOMMUNICATION - SERVICES


### 3. Load Cleaned DataFrame into DB Schema

Recall our DB Schema in Part 1:
- Table 1: ```Symbols```
    - Symbol (Type: ```Text```) **(Primary Key)**
    - Name (Type: ```Text```)
    - Industry (Type: ```Text```)
    
- Table 2: ```Membership```
    - (Symbol, ReportDate) **(Composite Primary Key)**
        - Symbol (Type: ```Text```)
        - ReportDate (Type: ```DateTime```)
    - Weightage (Type: ```Double```)

- Table 3: ```Attributes```
    - (Symbol, ReportDate) **(Composite Primary Key)**
        - Symbol (Type: ```Text```)
        - ReportDate (Type: ```DateTime```)
    - Equity Capital (In Rs.)
    - Free Float Market Capitalisation (Rs. Crores)
    - Beta (Type: ```Double```)	
    - R2 (Type: ```Double```)	
    - Volatility_Per (Type: ```Double```)	
    - Monthly Return (Type: ```Double```)	
    - Avg_Impact_Cost_percent (Type: ```Double```)

In order to populate data into our DB schema, here is what we will be doing:
- Create a new ```SQLite3``` DB called ```Nifty50.DB```
- Create SQL queries and execute them to create tables ```Symbols```, ```Membership``` and ```Attributes``` based on our DB schema
- Execute the abovementioned SQL queries
- Standardise the column names of our dataframe and SQL DB schema tables
- Partition our cleaned dataframe to match our DB tables
- Load them into our database afterwards
- **Any subsequent data analytics is performed by using SQL query to get data from our DB in the form of dataframes and then performing data analytics with it**

The reason why I chose ```SQLite3``` is as follows:
- Our raw dataframe is considered moderately small, and ```SQLite3``` is suitable for such low to moderate database traffic considering its a lightweight, file-based database engine
- Any new excel files can be first read as a pandas dataframe, before being cleaned appropriately and then populated into our DB schema easily, making it easily scalable
- It is also portable because anyone with the DB file can read it and access it easily as well, considering its serverless too
- The downsides would be that this may not be suitable for large scale applications BUT:
    - The nature of this dataset is more of a reference dataset rather than pricing tick dataset
    - If this were to be pricing dataset however, using databases like ```PostgreSQL``` would be better, but the drawback of it would be more maintenance and troublesome to configure. Also, a ticker plant would need to be configured for this, which is totally not necessary for our scenario here
    - Henceforth, using ```SQLite3``` here for our task is enough considering the nature of the dataset and size of it






In [3]:
# Create a new SQLite3 DB and cursor object for executing SQL queries
connection = sqlite3.connect('Nifty50.db')
cursor = connection.cursor()


# Define SQL Queries to create tables per our DB Schema
create_symbols_table = '''
CREATE TABLE IF NOT EXISTS Symbols (
    Symbol TEXT PRIMARY KEY,
    Name TEXT,
    Industry TEXT
);
'''

create_membership_table = '''
CREATE TABLE IF NOT EXISTS Membership (
    Symbol TEXT,
    ReportDate DATETIME,
    Weightage DOUBLE,
    PRIMARY KEY (Symbol, ReportDate),
    FOREIGN KEY (Symbol) REFERENCES Symbols(Symbol)
);
'''

create_attributes_table = '''
CREATE TABLE IF NOT EXISTS Attributes (
    Symbol TEXT,
    ReportDate DATETIME,
    Equity_Capital DOUBLE,
    Free_Float_Market_Capitalisation DOUBLE,
    Beta DOUBLE,
    R2 DOUBLE,
    Volatility_Per DOUBLE,
    Monthly_Return DOUBLE,
    Avg_Impact_Cost_Percent DOUBLE,
    PRIMARY KEY (Symbol, ReportDate),
    FOREIGN KEY (Symbol) REFERENCES Symbols(Symbol)
);
'''


# Execute the SQL queries to create tables
cursor.execute(create_symbols_table)
cursor.execute(create_membership_table)
cursor.execute(create_attributes_table)


# Commit the changes and close the connection
connection.commit()
connection.close()


# Standardise column names for cleaned dataframe before populating into our DB schema
correct_naming = {
    'Equity Capital (In Rs.)': 'Equity_Capital',
    'Free Float Market Capitalisation (Rs. Crores)': 'Free_Float_Market_Capitalisation',
    'Weightage ()': 'Weightage',
    'Monthly Return': 'Monthly_Return',
    'Avg_Impact_Cost_percent': 'Avg_Impact_Cost_Percent',
    'report_date': 'ReportDate'}
cleaned.rename(columns=correct_naming, inplace=True)


# Partition our cleaned tables and populate them in our schema later
# "Symbols" dataframe (previously created above) will be populated into our DB schema "Symbols" table
# Membership dataframe will be populated into our DB schema "Membership" table
engine = create_engine('sqlite:///Nifty50.db')

Membership = cleaned[['Symbol', 'ReportDate', 'Weightage']]
Attributes = cleaned[['Symbol', 'ReportDate', 'Equity_Capital', 'Free_Float_Market_Capitalisation', 
                       'Beta','R2', 'Volatility_Per', 'Monthly_Return', 'Avg_Impact_Cost_Percent']]

Symbols.to_sql('Symbols', con=engine, index=False, if_exists='replace')
Membership.to_sql('Membership', con=engine, index=False, if_exists='replace')
Attributes.to_sql('Attributes', con=engine, index=False, if_exists='replace')


6800

### 4. Calculate Simple moving average for Equity Capital of Tickers over past 6 months window

In [4]:
# Create a generic function
def SMA_EquityCapital(window_size):

    '''
    :window_size: Window size user provides which can be adjustable
    Returns a pandas dataframe showing SMA of equity capital of symbols per window size
    '''

    # Create SQL query
    connection = sqlite3.connect('Nifty50.db')
    query = f'''
        SELECT
            Symbol,
            ReportDate,
            Equity_Capital,
            AVG(Equity_Capital) OVER (PARTITION BY Symbol ORDER BY ReportDate ROWS BETWEEN {window_size - 1} 
                                      PRECEDING AND CURRENT ROW) AS SMA_Equity_Capital
        FROM Attributes
        WHERE ReportDate <= (
            SELECT MAX(ReportDate)
            FROM Attributes
            WHERE Symbol = Attributes.Symbol
        )
        ORDER BY Symbol, ReportDate
    '''

    # Execute SQL query and return result
    result = pd.read_sql_query(query, connection, parse_dates=['ReportDate'])
    return result


# Test it out
SMA_EquityCapital(window_size=6)


,Symbol,ReportDate,Equity_Capital,SMA_Equity_Capital
0,ABB,2008-01-01,423816750.0,4.238168e+08
1,ABB,2008-02-01,423816750.0,4.238168e+08
2,ABB,2008-03-01,423816750.0,4.238168e+08
3,ABB,2008-04-01,423816750.0,4.238168e+08
4,ABB,2008-05-01,423816750.0,4.238168e+08
...,...,...,...,...
6795,ZEEL,2019-05-01,960466500.0,9.604665e+08
6796,ZEEL,2019-06-01,960481765.0,9.604690e+08
6797,ZEEL,2019-07-01,960481765.0,9.604716e+08
6798,ZEEL,2019-08-01,960481765.0,9.604741e+08


### 5. Calculate year wise high and low Equity Capital for each stock.

In [5]:
# Create a generic function
def Year_HighLow_EquityCapital():
    
    '''
    Returns a pandas dataframe
    '''

    # Create SQL query
    connection = sqlite3.connect('Nifty50.db')

    # Execute SQL query and return result
    query = '''
        SELECT
            Symbol,
            strftime('%Y', ReportDate) AS Year,
            MAX(Equity_Capital) AS Max_Equity_Capital,
            MIN(Equity_Capital) AS Min_Equity_Capital
        FROM Attributes
        GROUP BY Symbol, Year
        ORDER BY Symbol, Year;
    '''
    
    # Execute SQL query and return results
    result = pd.read_sql_query(query, connection)
    return result


# Test it out
Year_HighLow_EquityCapital()


,Symbol,Year,Max_Equity_Capital,Min_Equity_Capital
0,ABB,2008,4.238168e+08,4.238168e+08
1,ABB,2009,4.238168e+08,4.238168e+08
2,ABB,2010,4.238168e+08,4.238168e+08
3,ACC,2008,1.876609e+09,1.876243e+09
4,ACC,2009,1.877109e+09,1.876817e+09
...,...,...,...,...
644,ZEEL,2015,9.604487e+08,9.604487e+08
645,ZEEL,2016,9.604487e+08,9.604487e+08
646,ZEEL,2017,9.604536e+08,9.604487e+08
647,ZEEL,2018,9.604665e+08,9.604536e+08


### 6. Find 2 tickers with highest positive correlation in prices movement and 2 tickers with highest inverse correlation

In [6]:
# Create a generic function
def correlation_tickers(input):

    '''
    :input: Input column for correlation analysis. Must match SQL column names in Attributes table
    Returns a pandas dataframe
    '''

    # Connect to the SQLite database (adjust the database name)
    connection = sqlite3.connect('Nifty50.db')

    # Execute SQL query and get results
    query = f'''
        SELECT
            Symbol,
            ReportDate,
            {input}
        FROM
            Attributes
    '''
    result = pd.read_sql_query(query, connection, parse_dates=['ReportDate'])

    # Pivot result and get correlation matrix
    pivot_result = result.pivot(index='ReportDate', columns='Symbol', values='Monthly_Return')
    corr_matrix = pivot_result.corr()

    # Get 2 tickers with highest positive correlation based on specified column
    Highest2 = list(corr_matrix.unstack().idxmax())

    # Get 2 tickers with highest inverse correlation based on specified column
    Lowest2 = list(corr_matrix.unstack().idxmax())

    # Return correlation result of pairs of tickers
    final =  pd.DataFrame({
        'Highest positive correlation 2 tickers': Highest2,
        'Highest inverse correlation 2 tickers': Lowest2
    })

    return final


# Test it out
correlation_tickers('Monthly_Return')


,Highest positive correlation 2 tickers,Highest inverse correlation 2 tickers
0,AMBUJACEM,AMBUJACEM
1,GLAXO,GLAXO
